In [1]:
!pip install nltk textblob transformers torch pyyaml


In [2]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [4]:
import os

folders = ["tool_agent", "tool_agent/logs"]
for f in folders:
    os.makedirs(f, exist_ok=True)


In [5]:
%%writefile tool_agent/config.yaml
constraints:
  dataset_size_weight: 0.3
  speed_weight: 0.25
  accuracy_weight: 0.25
  resource_weight: 0.2


Writing tool_agent/config.yaml


In [6]:
%%writefile tool_agent/tool_registry.py

TOOLS = {
    "vader": {
        "speed": 9,
        "accuracy": 6,
        "resource": 9,
        "offline": True,
        "description": "NLTK VADER sentiment analyzer"
    },
    "textblob": {
        "speed": 7,
        "accuracy": 7,
        "resource": 8,
        "offline": True,
        "description": "TextBlob polarity analyzer"
    },
    "huggingface": {
        "speed": 5,
        "accuracy": 9,
        "resource": 4,
        "offline": False,
        "description": "Transformer sentiment model"
    }
}


Writing tool_agent/tool_registry.py


In [7]:
%%writefile tool_agent/decision_engine.py

import yaml

def load_weights():
    with open("tool_agent/config.yaml") as f:
        return yaml.safe_load(f)["constraints"]

def score_tool(tool_meta, weights):
    return (
        tool_meta["speed"] * weights["speed_weight"]
        + tool_meta["accuracy"] * weights["accuracy_weight"]
        + tool_meta["resource"] * weights["resource_weight"]
    )

def select_best_tool(tools):
    weights = load_weights()
    scores = {}

    for name, meta in tools.items():
        scores[name] = score_tool(meta, weights)

    best_tool = max(scores, key=scores.get)
    return best_tool, scores


Writing tool_agent/decision_engine.py


In [36]:
%%writefile tool_agent/executor.py

from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import pipeline

def run_vader(texts):
    sia = SentimentIntensityAnalyzer()
    return [sia.polarity_scores(t)["compound"] for t in texts]

def run_textblob(texts):
    return [TextBlob(t).sentiment.polarity for t in texts]

def run_huggingface(texts):
    classifier = pipeline("sentiment-analysis")
    return classifier(texts)

def execute(tool, data):
    if tool == "vader":
        return run_vader(data)
    elif tool == "textblob":
        return run_textblob(data)
    elif tool == "huggingface":
        return run_huggingface(data)


Overwriting tool_agent/executor.py


In [32]:
%%writefile tool_agent/logger.py

from datetime import datetime

def log_decision(content):
    with open("tool_agent/logs/decisions.log", "a") as f:
        f.write(f"{datetime.now()} | {content}\n")


Overwriting tool_agent/logger.py


In [39]:
%%writefile tool_agent/agent.py

from tool_registry import TOOLS
from decision_engine import select_best_tool
from executor import execute, run_vader, run_textblob, run_huggingface
from logger import log_decision
import time
import warnings
import os
from transformers import logging as hf_logging
from contextlib import redirect_stdout, redirect_stderr
import io

# Silence warnings & HF logs
warnings.filterwarnings("ignore")
hf_logging.set_verbosity_error()
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

class ToolChoosingAgent:

    def silent_run(self, func, texts):
        f = io.StringIO()
        with redirect_stdout(f), redirect_stderr(f):
            return func(texts)

    def runtime(self, func, texts):
        start = time.time()
        self.silent_run(func, texts)
        return round(time.time() - start, 3)

    def accuracy(self, func, texts, labels):
        preds = self.silent_run(func, texts)
        correct = 0

        for pred, label in zip(preds, labels):
            if isinstance(pred, dict):
                sentiment = 1 if pred['label'] == 'POSITIVE' else -1
            else:
                sentiment = 1 if pred > 0 else -1

            if sentiment == label:
                correct += 1

        return round((correct / len(labels)) * 100, 2)

    def run_task(self, task, data):

        labeled_data = [
            ("This product is amazing!", 1),
            ("Very bad experience", -1),
            ("I love this service", 1),
            ("Worst purchase ever", -1)
        ]

        texts = [x[0] for x in labeled_data]
        labels = [x[1] for x in labeled_data]

        best_tool, scores = select_best_tool(TOOLS)

        log_decision(f"Task: {task}")
        log_decision(f"Scores: {scores}")
        log_decision(f"Chosen Tool: {best_tool}")

        print("\n================ TOOL EVALUATION REPORT ================\n")

        print(f"VADER        : {scores['vader']:.2f}")
        print(f"TextBlob     : {scores['textblob']:.2f}")
        print(f"HuggingFace  : {scores['huggingface']:.2f}")

        print("\n-------------------------------------------------------\n")
        print(f"Selected Tool : {best_tool.upper()}")
        print("Justification : Highest weighted performance score")

        print("\n================ PERFORMANCE METRICS ===================\n")
        print("Tool         Runtime (s)     Accuracy (%)")
        print("-----------------------------------------")

        print(f"VADER            {self.runtime(run_vader, texts):<6}         {self.accuracy(run_vader, texts, labels)}")
        print(f"TextBlob         {self.runtime(run_textblob, texts):<6}         {self.accuracy(run_textblob, texts, labels)}")
        print(f"HuggingFace      {self.runtime(run_huggingface, texts):<6}         {self.accuracy(run_huggingface, texts, labels)}")

        print("\n================ EXECUTION RESULTS =====================\n")

        results = execute(best_tool, data)

        for i, score in enumerate(results, 1):
            sentiment = "Positive" if score > 0 else "Negative"
            print(f"Review {i} : {score:>7}   ({sentiment})")

        return results


Overwriting tool_agent/agent.py


In [34]:
%%writefile tool_agent/main.py

from agent import ToolChoosingAgent

if __name__ == "__main__":
    data = [
        "This product is amazing!",
        "Very bad experience",
        "I love this service",
        "Worst purchase ever"
    ]

    agent = ToolChoosingAgent()
    output = agent.run_task("Sentiment Analysis", data)

    print(output)


Overwriting tool_agent/main.py


In [40]:
!python tool_agent/main.py



================ TOOL EVALUATION REPORT ================

VADER        : 5.55
TextBlob     : 5.10
HuggingFace  : 4.30

-------------------------------------------------------

Selected Tool : VADER
Justification : Highest weighted performance score

================ PERFORMANCE METRICS ===================

Tool         Runtime (s)     Accuracy (%)
-----------------------------------------
VADER            0.018          100.0
TextBlob         0.078          100.0
HuggingFace      1.083          100.0

================ EXECUTION RESULTS =====================

Review 1 :  0.6239   (Positive)
Review 2 : -0.5849   (Negative)
Review 3 :  0.6369   (Positive)
Review 4 : -0.6249   (Negative)
[0.6239, -0.5849, 0.6369, -0.6249]


# **Decision Insight**
- **VADER** is fastest and resource-efficient.
- **HuggingFace** is most accurate but slowest.
- **Agent selects** based on constraint priorities.

In [41]:
!cat tool_agent/logs/decisions.log


2026-02-04 05:34:08.695871 | Task: Sentiment Analysis
2026-02-04 05:34:08.695960 | Scores: {'vader': 5.55, 'textblob': 5.1, 'huggingface': 4.3}
2026-02-04 05:34:08.695998 | Chosen Tool: vader
2026-02-04 05:41:58.336425 | Task: Sentiment Analysis
2026-02-04 05:41:58.336538 | Scores: {'vader': 5.55, 'textblob': 5.1, 'huggingface': 4.3}
2026-02-04 05:41:58.336581 | Chosen Tool: vader
2026-02-04 05:42:40.559559 | Task: Sentiment Analysis
2026-02-04 05:42:40.559638 | Scores: {'vader': 5.55, 'textblob': 5.1, 'huggingface': 4.3}
2026-02-04 05:42:40.559666 | Chosen Tool: vader
2026-02-04 05:44:08.115639 | Task: Sentiment Analysis
2026-02-04 05:44:08.115740 | Scores: {'vader': 5.55, 'textblob': 5.1, 'huggingface': 4.3}
2026-02-04 05:44:08.115778 | Chosen Tool: vader
2026-02-04 05:44:31.305924 | Task: Sentiment Analysis
2026-02-04 05:44:31.306001 | Scores: {'vader': 5.55, 'textblob': 5.1, 'huggingface': 4.3}
2026-02-04 05:44:31.306029 | Chosen Tool: vader
2026-02-04 05:46:30.017432 | Task: Senti

In [42]:
!zip -r tool_agent.zip tool_agent


  adding: tool_agent/ (stored 0%)
  adding: tool_agent/agent.py (deflated 65%)
  adding: tool_agent/__pycache__/ (stored 0%)
  adding: tool_agent/__pycache__/tool_registry.cpython-312.pyc (deflated 23%)
  adding: tool_agent/__pycache__/executor.cpython-312.pyc (deflated 39%)
  adding: tool_agent/__pycache__/logger.cpython-312.pyc (deflated 29%)
  adding: tool_agent/__pycache__/decision_engine.cpython-312.pyc (deflated 35%)
  adding: tool_agent/__pycache__/agent.cpython-312.pyc (deflated 46%)
  adding: tool_agent/tool_registry.py (deflated 61%)
  adding: tool_agent/executor.py (deflated 58%)
  adding: tool_agent/main.py (deflated 37%)
  adding: tool_agent/logs/ (stored 0%)
  adding: tool_agent/logs/decisions.log (deflated 84%)
  adding: tool_agent/decision_engine.py (deflated 56%)
  adding: tool_agent/logger.py (deflated 28%)
  adding: tool_agent/config.yaml (deflated 31%)


In [43]:
from google.colab import files
files.download("tool_agent.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
%%capture output
!python tool_agent/main.py


In [45]:
with open("README.md", "w") as f:
    f.write("# Tool-Chosing Autonomous Agent – Output Report\n\n")
    f.write("```\n")
    f.write(output.stdout)
    f.write("\n```")


In [46]:
from google.colab import files
files.download("README.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>